# 描述性统计量

In [58]:
# 生成描述性统计量的代码（表1）
import pandas as pd
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox
# 读取用户上传的更新后的数据文件
file_path = '/Users/quanzaixuan/Desktop/Assignment/统计思想pre/data1.csv'
df= pd.read_csv(file_path, parse_dates=True, index_col=0)

dat = df.apply(pd.to_numeric, errors='coerce')
# 初始化结果表格
results = pd.DataFrame(columns=['Mean', 'Median', 'Min', 'Max', 'Std. Dev', 'Skewness', 'Kurtosis', 'JB p-value', 'Q(1) p-value', 'Q(10) p-value'])

# 计算描述性统计量和检验
for col in['RET','BM','TBL','LTY','NTIS','INFL','SVAR','DP','DY','DFR','DFY','LTR','EP']:
    series = df[col].dropna()  # 去除缺失值
    mean = series.mean()
    median = series.median()
    min_val = series.min()
    max_val = series.max()
    std_dev = series.std()
    skewness = series.skew()
    kurtosis = series.kurtosis()
    
    # Jarque-Bera 检验
    jb_stat, jb_pvalue = jarque_bera(series)
    
    # Ljung-Box Q 检验
    q1_stat, q1_pvalue = acorr_ljungbox(series, lags=[1], return_df=False)  # 滞后长度1
    q10_stat, q10_pvalue = acorr_ljungbox(series, lags=[10], return_df=False)  # 滞后长度10
    
    # 填充结果
    results.loc[col] = [mean, median, min_val, max_val, std_dev, skewness, kurtosis, jb_pvalue, q1_pvalue[0], q10_pvalue[0]]

# 输出结果
print(results)

          Mean    Median       Min       Max  Std. Dev  Skewness   Kurtosis  \
RET   0.006294  0.009288 -0.291524  0.412291  0.054225  0.327891   9.738663   
BM    0.562749  0.535383  0.120510  2.028478  0.265780  0.800912   1.456685   
TBL   0.033742  0.029000  0.000100  0.163000  0.030727  1.109733   1.380064   
LTY   0.050690  0.041900  0.016300  0.148200  0.027870  1.107032   0.651988   
NTIS  0.016446  0.016609 -0.055954  0.177040  0.025873  1.619087   7.896769   
INFL  0.002414  0.002402 -0.020548  0.058824  0.005285  1.088402  14.031294   
SVAR  0.002838  0.001249  0.000072  0.070945  0.005697  5.843514  44.498046   
DP    1.395922  1.197344 -1.148281  4.083117  1.401028  0.112531  -1.216368   
DY    0.003975  0.004947 -0.093526  0.058424  0.011108 -2.042930  16.142617   
DFR   0.045913  0.038200 -0.107100  0.204700  0.036221  0.889594   2.038943   
DFY   0.011219  0.009000  0.003200  0.056400  0.006867  2.516416   9.133416   
LTR   0.004777  0.003050 -0.112400  0.152300  0.0245

## OLS预测

In [2]:
# Combined OLS with rolling window regression for single-variable prediction, store residuals and predictions
import pandas as pd
from sklearn.linear_model import LinearRegression
# Load data
data_path = '/Users/quanzaixuan/Desktop/Assignment/统计思想pre/data1.csv'
data = pd.read_csv(data_path)
# Convert 'yyyymm' to datetime format and set as index
data['yyyymm'] = pd.to_datetime(data['yyyymm'])
data.set_index('yyyymm', inplace=True)
# Define variables and split into train/test datasets
variables = ['BM', 'TBL', 'LTY', 'NTIS', 'INFL', 'SVAR', 'DP', 'DY', 'DFR', 'DFY', 'LTR', 'EP']
train_data = data.loc['1927-01-01':'1946-12-31']
test_data = data.loc['1947-01-01':]
# Initialize storage for rolling residuals and predictions
rolling_residuals = {var: [] for var in variables}
rolling_predictions = {var: [] for var in variables}
# Perform rolling OLS regression
for var in variables:
    for i in range(len(test_data)):
        # Define the rolling training data up to period k+i
        rolling_train_data = data.iloc[:len(train_data) + i]
        X_train = rolling_train_data[[var]]
        y_train = rolling_train_data['RET']
        # Define the next period (k+1) test data
        X_test = test_data[[var]].iloc[i:i+1]
        y_test = test_data['RET'].iloc[i:i+1]
        # Fit the model
        model = LinearRegression()
        model.fit(X_train, y_train)
        # Predict on test set
        y_pred = model.predict(X_test)
        # Store the residuals and predictions
        residual = y_test.values[0] - y_pred[0]
        rolling_residuals[var].append(residual)
        rolling_predictions[var].append(y_pred[0])
# Convert rolling residuals and predictions to DataFrames
rolling_residuals_df = pd.DataFrame(rolling_residuals, index=test_data.index[:len(test_data)])
rolling_predictions_df = pd.DataFrame(rolling_predictions, index=test_data.index[:len(test_data)])

# Display rolling residuals and predictions
rolling_residuals_df.head()




,BM,TBL,LTY,NTIS,INFL,SVAR,DP,DY,DFR,DFY,LTR,EP
yyyymm,,,,,,,,,,,,
1947-01-01,0.016782,0.017252,0.013079,0.017595,0.017773,0.011864,0.028049,0.017595,0.015437,0.013252,0.020787,0.033603
1947-02-01,-0.019240,-0.018951,-0.023180,-0.018927,-0.018421,-0.025280,-0.002424,-0.018590,-0.022876,-0.022969,-0.017738,0.003815
1947-03-01,-0.020144,-0.021424,-0.025562,-0.021203,-0.033974,-0.026364,-0.003119,-0.021089,-0.025335,-0.025208,-0.020143,0.005795
1947-04-01,-0.042288,-0.044362,-0.048090,-0.044082,-0.043841,-0.049411,-0.019093,-0.042761,-0.043482,-0.047914,-0.038214,-0.012380
1947-05-01,-0.011619,-0.013848,-0.017374,-0.014852,-0.013358,-0.018566,0.007946,-0.012154,-0.018568,-0.017140,-0.013753,0.018284


In [3]:
# Adjust column names for residuals and predictions

# Rename columns for residuals and predictions
rolling_residuals_df.columns = [f"ols_{var}_resid" for var in variables]
rolling_predictions_df.columns = [f"ols_{var}_pred" for var in variables]

# Display the updated dataframes
rolling_residuals_df.head()




,ols_BM_resid,ols_TBL_resid,ols_LTY_resid,ols_NTIS_resid,ols_INFL_resid,ols_SVAR_resid,ols_DP_resid,ols_DY_resid,ols_DFR_resid,ols_DFY_resid,ols_LTR_resid,ols_EP_resid
yyyymm,,,,,,,,,,,,
1947-01-01,0.016782,0.017252,0.013079,0.017595,0.017773,0.011864,0.028049,0.017595,0.015437,0.013252,0.020787,0.033603
1947-02-01,-0.019240,-0.018951,-0.023180,-0.018927,-0.018421,-0.025280,-0.002424,-0.018590,-0.022876,-0.022969,-0.017738,0.003815
1947-03-01,-0.020144,-0.021424,-0.025562,-0.021203,-0.033974,-0.026364,-0.003119,-0.021089,-0.025335,-0.025208,-0.020143,0.005795
1947-04-01,-0.042288,-0.044362,-0.048090,-0.044082,-0.043841,-0.049411,-0.019093,-0.042761,-0.043482,-0.047914,-0.038214,-0.012380
1947-05-01,-0.011619,-0.013848,-0.017374,-0.014852,-0.013358,-0.018566,0.007946,-0.012154,-0.018568,-0.017140,-0.013753,0.018284


## 选择最优带宽
### 选择时间权重最优带宽

In [4]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Define the Laplace kernel function
def laplace_kernel(u):
    return np.exp(-np.abs(u)) / 2

# Function to calculate time weights based on Laplace kernel
def calculate_time_weights(t, s, h):
    u = (t - s) / h
    return laplace_kernel(u)

# Function to perform weighted regression and return predictions
def weighted_regression(X_train, y_train, X_test, time_weights):
    model = LinearRegression()
    # Apply weights to the regression
    model.fit(X_train, y_train, sample_weight=time_weights)
    return model.predict(X_test)

# Cross-validation to select optimal time bandwidth
def cross_validate_time_bandwidth(data, var, bandwidths, p=5):
    # Split the dataset into train and test sets
    train_data = data.loc['1927-01-01':'1946-12-31']
    test_data = data.loc['1947-01-01':]

    best_bandwidth = None
    best_mse = float('inf')

    # Loop over each candidate bandwidth
    for h in bandwidths:
        mse_list = []

        # Perform leave-p-out cross-validation
        for i in range(p, len(train_data)):
            X_train = train_data[[var]].iloc[:i]
            y_train = train_data['RET'].iloc[:i]
            X_val = train_data[[var]].iloc[i:i+1]
            y_val = train_data['RET'].iloc[i:i+1]

            # Calculate time weights
            t = np.arange(i)
            s = np.array([i])
            time_weights = calculate_time_weights(t, s, h)

            # Perform weighted regression
            y_pred = weighted_regression(X_train, y_train, X_val, time_weights)

            # Calculate validation MSE
            mse = mean_squared_error(y_val, y_pred)
            mse_list.append(mse)

        avg_mse = np.mean(mse_list)
        if avg_mse < best_mse:
            best_mse = avg_mse
            best_bandwidth = h

    return best_bandwidth, best_mse



In [5]:
# Example usage
bandwidths = np.linspace(0.1, 10, 50)  # Candidate bandwidths for Laplace kernel
best_h, best_mse = cross_validate_time_bandwidth(data, 'LTR', bandwidths)

best_h, best_mse  # Output the best bandwidth and corresponding MSE



(10.0, 0.009102506627779335)

In [6]:
#为其他变量也选择相应的带宽
best_bandwidths_time = {}
variables = ['BM', 'LTY', 'NTIS', 'INFL', 'SVAR', 'DP', 'DY', 'DFR', 'DFY', 'LTR', 'EP']
for var in variables:
    best_h, best_mse = cross_validate_time_bandwidth(train_data, var, bandwidths)
    best_bandwidths_time[var] = best_h

best_bandwidths_time  # Output the best bandwidths for each variable

{'BM': 10.0,
 'LTY': 10.0,
 'NTIS': 10.0,
 'INFL': 10.0,
 'SVAR': 10.0,
 'DP': 0.5040816326530613,
 'DY': 10.0,
 'DFR': 10.0,
 'DFY': 10.0,
 'LTR': 10.0,
 'EP': 1.1102040816326533}

### 选择稳健权重最优带宽

In [7]:
# Define the weight function based on OLS residuals (for robust regression)
def calculate_residual_weights(residuals, h):
    """Calculate robust weights based on residuals and bandwidth h."""
    return np.exp(-np.abs(residuals / h)) / 2

# Function to perform cross-validation for optimal residual weight bandwidth selection using train_data only
def cross_validate_residual_bandwidth(train_data, var, bandwidths, p=5):
    best_bandwidth = None
    best_mse = float('inf')

    # Loop over each candidate bandwidth
    for h in bandwidths:
        mse_list = []

        # Perform leave-p-out cross-validation
        for i in range(p, len(train_data)):
            X_train = train_data[[var]].iloc[:i]
            y_train = train_data['RET'].iloc[:i]
            X_val = train_data[[var]].iloc[i:i+1]
            y_val = train_data['RET'].iloc[i:i+1]

            # First fit OLS to get residuals
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_train_pred = model.predict(X_train)
            residuals = y_train - y_train_pred

            # Calculate robust weights based on OLS residuals
            residual_weights = calculate_residual_weights(residuals, h)

            # Perform weighted regression with residual weights
            model.fit(X_train, y_train, sample_weight=residual_weights)
            y_pred = model.predict(X_val)

            # Calculate validation MSE
            mse = mean_squared_error(y_val, y_pred)
            mse_list.append(mse)

        avg_mse = np.mean(mse_list)
        if avg_mse < best_mse:
            best_mse = avg_mse
            best_bandwidth = h

    return best_bandwidth, best_mse

# Example usage for 'BM' variable using train_data
bandwidths = np.linspace(0.1, 10, 50)  # Candidate bandwidths for robust regression
best_h_residual, best_mse_residual = cross_validate_residual_bandwidth(train_data, 'BM', bandwidths)

best_h_residual, best_mse_residual  # Output the best residual weight bandwidth and corresponding MSE


(10.0, 0.008150433634071882)

In [8]:
#为其他变量也选择相应的带宽
best_bandwidths_residual = {}
variables = ['BM', 'LTY', 'NTIS', 'INFL', 'SVAR', 'DP', 'DY', 'DFR', 'DFY', 'LTR', 'EP']
for var in variables:
    best_h, best_mse = cross_validate_residual_bandwidth(train_data, var, bandwidths)
    best_bandwidths_residual[var] = best_h

best_bandwidths_residual  # Output the best bandwidths for each variable

{'BM': 10.0,
 'LTY': 0.5040816326530613,
 'NTIS': 10.0,
 'INFL': 0.1,
 'SVAR': 10.0,
 'DP': 0.1,
 'DY': 10.0,
 'DFR': 10.0,
 'DFY': 10.0,
 'LTR': 10.0,
 'EP': 10.0}

## TRWLS 预测

In [9]:
# Define the TRWLS regression process using both time weights and residual weights
def trwls_predict(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual):
    predictions = []

    # Rolling prediction (same as OLS: use data until k to predict k+1)
    for i in range(len(test_data)):
        X_train = train_data[[var]].iloc[:len(train_data) + i]
        y_train = train_data['RET'].iloc[:len(train_data) + i]
        X_test = test_data[[var]].iloc[i:i+1]
        
        # Time weights (using Laplace kernel and best time bandwidth)
        t = np.arange(len(X_train))
        s = np.array([len(X_train)])
        time_weights = calculate_time_weights(t, s, best_bandwidth_time)

        # First OLS to get residuals
        model_ols = LinearRegression()
        model_ols.fit(X_train, y_train)
        y_train_pred = model_ols.predict(X_train)
        residuals = y_train - y_train_pred

        # Residual weights (based on OLS residuals and best residual bandwidth)
        residual_weights = calculate_residual_weights(residuals, best_bandwidth_residual)

        # Final combined weights: time weights * residual weights
        combined_weights = time_weights * residual_weights

        # Perform weighted regression using TRWLS
        model_trwls = LinearRegression()
        model_trwls.fit(X_train, y_train, sample_weight=combined_weights)
        y_pred = model_trwls.predict(X_test)

        # Store prediction
        predictions.append(y_pred[0])

    return predictions

In [10]:
# Example usage for 'BM' variable using stored best bandwidths for time and residuals
var = 'BM'
best_bandwidth_time = best_bandwidths_time[var]
best_bandwidth_residual = best_bandwidths_residual[var]

# Perform TRWLS predictions
trwls_predictions = trwls_predict(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual)

# Output the predictions
trwls_predictions[:5]  # Display the first few predictions for reference

[-0.003662638279096367,
 -0.004107049154393194,
 -0.008334106397125514,
 -0.010498405109265821,
 -0.010978539927018971]

In [11]:
# Initialize a DataFrame to store TRWLS predictions for all variables
trwls_pred_df = pd.DataFrame(index=test_data.index)

# Loop through all variables and perform TRWLS prediction for each
for var in variables:
    best_bandwidth_time = best_bandwidths_time[var]
    best_bandwidth_residual = best_bandwidths_residual[var]

    # Perform TRWLS predictions for the variable
    trwls_predictions = trwls_predict(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual)

    # Store the predictions in the DataFrame with the appropriate column name
    trwls_pred_df[f"{var}_trwls_pred"] = trwls_predictions

# Display the TRWLS predictions DataFrame
trwls_pred_df.head()


,BM_trwls_pred,LTY_trwls_pred,NTIS_trwls_pred,INFL_trwls_pred,SVAR_trwls_pred,DP_trwls_pred,DY_trwls_pred,DFR_trwls_pred,DFY_trwls_pred,LTR_trwls_pred,EP_trwls_pred
yyyymm,,,,,,,,,,,
1947-01-01,-0.003663,-0.001459,-0.000752,0.014951,0.006641,0.003314,-0.003018,-0.009540,-0.004678,-0.010276,0.088467
1947-02-01,-0.004107,-0.001459,0.002067,0.014951,0.012620,-0.042665,-0.003039,-0.003652,-0.004200,-0.004174,0.154493
1947-03-01,-0.008334,-0.001404,0.000266,-0.003756,0.004245,-0.057471,-0.003010,-0.003652,-0.002767,-0.004400,0.200784
1947-04-01,-0.010498,-0.001624,-0.000374,0.014951,0.005076,-0.113649,-0.005483,-0.016954,-0.001334,-0.017282,0.250604
1947-05-01,-0.010979,-0.001569,0.010331,0.014951,0.003175,-0.086827,-0.005448,-0.001472,-0.000856,-0.001462,0.252946


## 评估样本外预测表现

In [12]:
from sklearn.metrics import r2_score
import numpy as np
from scipy import stats

# Function to calculate R-squared for OLS and TRWLS predictions
def calculate_r2(y_true, y_pred):
    return r2_score(y_true, y_pred)

# Function to perform Clark-West (CW) test
def clark_west_test(y_true, y_pred_ols, y_pred_trwls):
    # Calculate the prediction errors for both models
    error_ols = y_true - y_pred_ols
    error_trwls = y_true - y_pred_trwls

    # Calculate the squared prediction errors
    error_diff = (error_ols ** 2) - (error_trwls ** 2)
    adjusted_error_diff = error_diff + ((y_pred_trwls - y_pred_ols) ** 2)

    # Perform one-sample t-test
    cw_stat, cw_p_value = stats.ttest_1samp(adjusted_error_diff, 0)

    return cw_stat, cw_p_value

# Initialize storage for results
r2_ols_results = {}
r2_trwls_results = {}
cw_test_results = {}

# Loop through all variables and compute R-squared and CW test
for var in variables:
    # Get the true values (RET) from the test data
    y_true = test_data['RET'].values

    # Get the OLS predictions
    y_pred_ols = rolling_predictions_df[f"ols_{var}_pred"].values

    # Get the TRWLS predictions
    y_pred_trwls = trwls_pred_df[f"{var}_trwls_pred"].values

    # Calculate R-squared for OLS and TRWLS
    r2_ols = calculate_r2(y_true, y_pred_ols)
    r2_trwls = calculate_r2(y_true, y_pred_trwls)
    
    # Store the R-squared results
    r2_ols_results[var] = r2_ols
    r2_trwls_results[var] = r2_trwls

    # Perform CW test
    cw_stat, cw_p_value = clark_west_test(y_true, y_pred_ols, y_pred_trwls)
    cw_test_results[var] = {"CW_stat": cw_stat, "CW_p_value": cw_p_value}

# Display the R-squared results and CW test results
r2_ols_results, r2_trwls_results, cw_test_results

# Convert the results to DataFrames for better display
r2_ols_df = pd.DataFrame(r2_ols_results, index=["R2_OLS"]).T
r2_trwls_df = pd.DataFrame(r2_trwls_results, index=["R2_TRWLS"]).T
cw_test_df = pd.DataFrame(cw_test_results).T

# Concatenate the DataFrames
results_df = pd.concat([r2_ols_df, r2_trwls_df, cw_test_df], axis=1)
results_df



,R2_OLS,R2_TRWLS,CW_stat,CW_p_value
BM,-0.005668,-0.168634,-2.201577,2.795558e-02
LTY,-0.010613,-0.589546,-2.084951,3.736315e-02
NTIS,-0.004475,-1.273895,-1.579220,1.146470e-01
INFL,-0.009232,-0.012766,2.433455,1.515475e-02
SVAR,0.063464,-0.119907,1.877644,6.076229e-02
DP,-0.291803,-6537.649059,-5.747164,1.252755e-08
DY,-0.003296,-0.059517,-0.890195,3.736059e-01
DFR,-0.019177,-7.742377,-4.482409,8.361135e-06
DFY,-0.004726,-0.248264,1.135744,2.563746e-01
LTR,-0.013774,-1.967906,-3.884624,1.102155e-04


In [13]:
# Split the test data into two periods: 1947-1983 and 1984-2019
test_data_1 = test_data.loc['1947-01-01':'1983-12-31']
test_data_2 = test_data.loc['1984-01-01':'2019-12-31']

# Function to calculate R-squared and CW test for a specific period
def evaluate_model_performance(test_data_period, var, rolling_predictions_df, trwls_pred_df):
    y_true = test_data_period['RET'].values
    y_pred_ols = rolling_predictions_df[f"ols_{var}_pred"].loc[test_data_period.index].values
    y_pred_trwls = trwls_pred_df[f"{var}_trwls_pred"].loc[test_data_period.index].values
    
    # Calculate R-squared for OLS and TRWLS
    r2_ols = calculate_r2(y_true, y_pred_ols)
    r2_trwls = calculate_r2(y_true, y_pred_trwls)
    
    # Perform CW test
    cw_stat, cw_p_value = clark_west_test(y_true, y_pred_ols, y_pred_trwls)

    return r2_ols, r2_trwls, cw_stat, cw_p_value

# Initialize storage for results
r2_ols_results_1 = {}
r2_trwls_results_1 = {}
cw_test_results_1 = {}

r2_ols_results_2 = {}
r2_trwls_results_2 = {}
cw_test_results_2 = {}

# Loop through all variables and evaluate performance for both periods
for var in variables:
    # Period 1: 1947-1983
    r2_ols_1, r2_trwls_1, cw_stat_1, cw_p_value_1 = evaluate_model_performance(test_data_1, var, rolling_predictions_df, trwls_pred_df)
    r2_ols_results_1[var] = r2_ols_1
    r2_trwls_results_1[var] = r2_trwls_1
    cw_test_results_1[var] = {"CW_stat": cw_stat_1, "CW_p_value": cw_p_value_1}
    
    # Period 2: 1984-2019
    r2_ols_2, r2_trwls_2, cw_stat_2, cw_p_value_2 = evaluate_model_performance(test_data_2, var, rolling_predictions_df, trwls_pred_df)
    r2_ols_results_2[var] = r2_ols_2
    r2_trwls_results_2[var] = r2_trwls_2
    cw_test_results_2[var] = {"CW_stat": cw_stat_2, "CW_p_value": cw_p_value_2}

# Output the results for both periods
(r2_ols_results_1, r2_trwls_results_1, cw_test_results_1), (r2_ols_results_2, r2_trwls_results_2, cw_test_results_2)



(({'BM': -0.0026930234057651603,
   'LTY': -0.017212442097554126,
   'NTIS': 0.008210272473298308,
   'INFL': -0.015530262752474489,
   'SVAR': 0.004959181272883639,
   'DP': -0.5830367871324305,
   'DY': -0.0032036028596558186,
   'DFR': -0.01348981798379656,
   'DFY': -0.008346596381463067,
   'LTR': 0.02699300754745304,
   'EP': -0.22443744167049218},
  {'BM': -0.1683527425245741,
   'LTY': -0.5873198553248162,
   'NTIS': -0.1592427395660696,
   'INFL': -0.007675322262889583,
   'SVAR': -0.021248159968980884,
   'DP': -2090.626573290667,
   'DY': -0.05207522344308191,
   'DFR': -8.02148232084323,
   'DFY': -0.2056985503595199,
   'LTR': -1.2443937672583791,
   'EP': -998.6779116821085},
  {'BM': {'CW_stat': -1.2196758553451106, 'CW_p_value': 0.22323672458697325},
   'LTY': {'CW_stat': -1.991270635252197, 'CW_p_value': 0.04706484461946905},
   'NTIS': {'CW_stat': 1.0242854898214515, 'CW_p_value': 0.3062594368169186},
   'INFL': {'CW_stat': 2.5238273366620416, 'CW_p_value': 0.01195709

In [14]:
# Convert the results to DataFrames for better display
r2_ols_df_1 = pd.DataFrame(r2_ols_results_1, index=["R2_OLS_1"]).T
r2_trwls_df_1 = pd.DataFrame(r2_trwls_results_1, index=["R2_TRWLS_1"]).T
cw_test_df_1 = pd.DataFrame(cw_test_results_1).T

r2_ols_df_2 = pd.DataFrame(r2_ols_results_2, index=["R2_OLS_2"]).T
r2_trwls_df_2 = pd.DataFrame(r2_trwls_results_2, index=["R2_TRWLS_2"]).T
cw_test_df_2 = pd.DataFrame(cw_test_results_2).T

# Concatenate the DataFrames for both periods
result_df_Rsqua=pd.concat([r2_ols_df_1, r2_trwls_df_1, cw_test_df_1, r2_ols_df_2, r2_trwls_df_2, cw_test_df_2],axis=1)
result_df_Rsqua

,R2_OLS_1,R2_TRWLS_1,CW_stat,CW_p_value,R2_OLS_2,R2_TRWLS_2,CW_stat,CW_p_value
BM,-0.002693,-0.168353,-1.219676,2.232367e-01,-0.009213,-0.169884,-1.964576,0.050105
LTY,-0.017212,-0.587320,-1.991271,4.706484e-02,-0.005516,-0.592912,-1.043476,0.297313
NTIS,0.008210,-0.159243,1.024285,3.062594e-01,-0.016788,-2.282432,-1.982423,0.048066
INFL,-0.015530,-0.007675,2.523827,1.195710e-02,-0.004405,-0.018228,0.938208,0.348663
SVAR,0.004959,-0.021248,0.802350,4.227808e-01,0.115498,-0.209958,1.757106,0.079610
DP,-0.583037,-2090.626573,-8.413722,5.524102e-16,-0.029903,-10559.149418,-2.704541,0.007111
DY,-0.003204,-0.052075,-0.357627,7.207926e-01,-0.004235,-0.067141,-0.868418,0.385649
DFR,-0.013490,-8.021482,-3.014777,2.719419e-03,-0.025183,-7.497787,-3.314912,0.000994
DFY,-0.008347,-0.205699,2.384382,1.752780e-02,-0.002314,-0.287768,-0.417079,0.676828
LTR,0.026993,-1.244394,-0.516501,6.057623e-01,-0.051454,-2.623812,-4.182391,0.000035


## 经济价值检验

In [15]:
# Function to calculate portfolio returns, cumulative gain (CG), and certainty equivalent return (CER)
def portfolio_performance(y_pred, y_true, gamma=3):
    # Portfolio return (assuming investment weight is proportional to predicted returns)
    portfolio_returns = y_pred * y_true

    # Cumulative Gain (CG)
    cumulative_gain = np.prod(1 + portfolio_returns) - 1

    # Expected return and variance
    expected_return = np.mean(portfolio_returns)
    return_variance = np.var(portfolio_returns)

    # Certainty Equivalent Return (CER)
    cer = expected_return - (gamma / 2) * return_variance

    return cumulative_gain, cer

# Initialize storage for performance results
performance_results = {
    "Variable": [],
    "OLS_CG": [],
    "OLS_CER": [],
    "TRWLS_CG": [],
    "TRWLS_CER": []
}

# Loop through each variable and calculate performance for both OLS and TRWLS
for var in variables:
    # Get true values (RET) and OLS/TRWLS predicted values
    y_true = test_data['RET'].values
    y_pred_ols = rolling_predictions_df[f"ols_{var}_pred"].values
    y_pred_trwls = trwls_pred_df[f"{var}_trwls_pred"].values

    # Calculate OLS portfolio performance
    ols_cg, ols_cer = portfolio_performance(y_pred_ols, y_true, gamma=3)

    # Calculate TRWLS portfolio performance
    trwls_cg, trwls_cer = portfolio_performance(y_pred_trwls, y_true, gamma=3)

    # Store results
    performance_results["Variable"].append(var)
    performance_results["OLS_CG"].append(ols_cg)
    performance_results["OLS_CER"].append(ols_cer)
    performance_results["TRWLS_CG"].append(trwls_cg)
    performance_results["TRWLS_CER"].append(trwls_cer)

# Convert results to a DataFrame for better readability
performance_df = pd.DataFrame(performance_results)

# Display the performance results DataFrame
performance_df


,Variable,OLS_CG,OLS_CER,TRWLS_CG,TRWLS_CER
0,BM,0.061405,0.000068,0.083048,0.000090
1,LTY,0.024234,0.000027,-0.108371,-0.000132
2,NTIS,0.056033,0.000062,0.302917,0.000296
3,INFL,0.031947,0.000036,0.086027,0.000094
4,SVAR,0.117974,0.000127,0.459408,0.000407
5,DP,-0.045526,-0.000054,-1.000000,-0.047612
6,DY,0.032932,0.000037,-0.017620,-0.000020
7,DFR,0.039382,0.000044,-0.214118,-0.000298
8,DFY,0.037404,0.000042,0.106676,0.000114
9,LTR,0.080453,0.000088,0.203491,0.000201


In [21]:
# Function to calculate portfolio performance (CG and CER) for a specific time period
def evaluate_portfolio_performance_for_period(test_data_period, var, rolling_predictions_df, trwls_pred_df, gamma=3):
    y_true = test_data_period['RET'].values
    y_pred_ols = rolling_predictions_df[f"ols_{var}_pred"].loc[test_data_period.index].values
    y_pred_trwls = trwls_pred_df[f"{var}_trwls_pred"].loc[test_data_period.index].values

    # Calculate OLS portfolio performance
    ols_cg, ols_cer = portfolio_performance(y_pred_ols, y_true, gamma)

    # Calculate TRWLS portfolio performance
    trwls_cg, trwls_cer = portfolio_performance(y_pred_trwls, y_true, gamma)

    return ols_cg, ols_cer, trwls_cg, trwls_cer

# Initialize storage for performance results for both periods
performance_results_period_1 = {
    "Variable": [],
    "OLS_CG_1": [],
    "OLS_CER_1": [],
    "TRWLS_CG_1": [],
    "TRWLS_CER_1": []
}

performance_results_period_2 = {
    "Variable": [],
    "OLS_CG_2": [],
    "OLS_CER_2": [],
    "TRWLS_CG_2": [],
    "TRWLS_CER_2": []
}

# Loop through each variable and calculate performance for both periods
for var in variables:
    # Period 1: 1947-1983
    ols_cg_1, ols_cer_1, trwls_cg_1, trwls_cer_1 = evaluate_portfolio_performance_for_period(test_data_1, var, rolling_predictions_df, trwls_pred_df, gamma=3)
    performance_results_period_1["Variable"].append(var)
    performance_results_period_1["OLS_CG_1"].append(ols_cg_1)
    performance_results_period_1["OLS_CER_1"].append(ols_cer_1)
    performance_results_period_1["TRWLS_CG_1"].append(trwls_cg_1)
    performance_results_period_1["TRWLS_CER_1"].append(trwls_cer_1)

    # Period 2: 1984-2019
    ols_cg_2, ols_cer_2, trwls_cg_2, trwls_cer_2 = evaluate_portfolio_performance_for_period(test_data_2, var, rolling_predictions_df, trwls_pred_df, gamma=3)
    performance_results_period_2["Variable"].append(var)
    performance_results_period_2["OLS_CG_2"].append(ols_cg_2)
    performance_results_period_2["OLS_CER_2"].append(ols_cer_2)
    performance_results_period_2["TRWLS_CG_2"].append(trwls_cg_2)
    performance_results_period_2["TRWLS_CER_2"].append(trwls_cer_2)

# Convert results to DataFrames for better readability
performance_df_period_1 = pd.DataFrame(performance_results_period_1)
performance_df_period_2 = pd.DataFrame(performance_results_period_2)

# Merge performance_df and performance_df_period_1,performance_df_perirod_2
performance_df = pd.merge(performance_df, performance_df_period_1, on="Variable")
performance_df=pd.merge(performance_df, performance_df_period_2, on="Variable")
performance_df

,Variable,OLS_CG,OLS_CER,TRWLS_CG,TRWLS_CER,OLS_CG_1_x,OLS_CER_1_x,TRWLS_CG_1_x,TRWLS_CER_1_x,OLS_CG_1_y,OLS_CER_1_y,TRWLS_CG_1_y,TRWLS_CER_1_y,OLS_CG_2,OLS_CER_2,TRWLS_CG_2,TRWLS_CER_2
0,BM,0.061405,0.000068,0.083048,0.000090,0.018543,0.000041,0.000168,-1.666353e-07,0.018543,0.000041,0.000168,-1.666353e-07,0.042082,0.000095,0.082866,0.000183
1,LTY,0.024234,0.000027,-0.108371,-0.000132,0.005215,0.000012,-0.039079,-9.120024e-05,0.005215,0.000012,-0.039079,-9.120024e-05,0.018920,0.000043,-0.072109,-0.000175
2,NTIS,0.056033,0.000062,0.302917,0.000296,0.021355,0.000048,0.086224,1.849871e-04,0.021355,0.000048,0.086224,1.849871e-04,0.033953,0.000077,0.199493,0.000411
3,INFL,0.031947,0.000036,0.086027,0.000094,0.011978,0.000027,0.039680,8.741975e-05,0.011978,0.000027,0.039680,8.741975e-05,0.019733,0.000045,0.044578,0.000101
4,SVAR,0.117974,0.000127,0.459408,0.000407,0.028913,0.000064,0.066903,1.451970e-04,0.028913,0.000064,0.066903,1.451970e-04,0.086558,0.000191,0.367892,0.000676
5,DP,-0.045526,-0.000054,-1.000000,-0.047612,-0.048707,-0.000113,-0.990574,-1.611267e-02,-0.048707,-0.000113,-0.990574,-1.611267e-02,0.003345,0.000008,-1.000000,-0.079584
6,DY,0.032932,0.000037,-0.017620,-0.000020,0.013032,0.000029,-0.007349,-1.663197e-05,0.013032,0.000029,-0.007349,-1.663197e-05,0.019643,0.000045,-0.010347,-0.000024
7,DFR,0.039382,0.000044,-0.214118,-0.000298,0.021974,0.000049,0.043165,7.246619e-05,0.021974,0.000049,0.043165,7.246619e-05,0.017033,0.000039,-0.246637,-0.000678
8,DFY,0.037404,0.000042,0.106676,0.000114,0.016094,0.000036,0.074893,1.611369e-04,0.016094,0.000036,0.074893,1.611369e-04,0.020972,0.000048,0.029568,0.000065
9,LTR,0.080453,0.000088,0.203491,0.000201,0.047349,0.000104,0.212242,4.272493e-04,0.047349,0.000104,0.212242,4.272493e-04,0.031608,0.000071,-0.007218,-0.000031


## 考虑其他的预测模型

In [27]:
from sklearn.linear_model import HuberRegressor
import pandas as pd

# Function to perform Huber regression with forward step prediction
def huber_predict_forward(train_data, test_data, var):
    n_test = len(test_data)
    
    # Prepare the data
    X_test = test_data[[var]].values

    huber_predictions = []

    # Perform forward step (one-step ahead) prediction
    for i in range(n_test):
        # Use data up to k for training and predict k+1
        X_train_step = train_data[[var]].iloc[:len(train_data) + i].values
        y_train_step = train_data['RET'].iloc[:len(train_data) + i].values
        
        # Initialize the Huber Regressor
        model = HuberRegressor()

        # Fit the model on the rolling training data
        model.fit(X_train_step, y_train_step)

        # Predict the next step
        pred = model.predict(X_test[i].reshape(1, -1))
        huber_predictions.append(pred[0])

    return huber_predictions

# Initialize a DataFrame to store Huber predictions for all variables
huber_pred_df = pd.DataFrame(index=test_data.index)

# Loop through all variables and perform Huber prediction with forward step for each
for var in variables:
    huber_predictions = huber_predict_forward(train_data, test_data, var)
    huber_pred_df[f"{var}_huber_pred"] = huber_predictions

# Display the Huber predictions DataFrame
huber_pred_df.head()

,BM_huber_pred,LTY_huber_pred,NTIS_huber_pred,INFL_huber_pred,SVAR_huber_pred,DP_huber_pred,DY_huber_pred,DFR_huber_pred,DFY_huber_pred,LTR_huber_pred,EP_huber_pred
yyyymm,,,,,,,,,,,
1947-01-01,0.006134,0.007745,0.006297,0.006385,0.011181,-0.003628,0.006672,0.007410,0.015677,0.003697,-0.001587
1947-02-01,0.005895,0.007745,0.006438,0.006385,0.013534,-0.008244,0.006677,0.008995,0.015581,0.005594,-0.005756
1947-03-01,0.003620,0.007766,0.006348,0.008367,0.010237,-0.009731,0.006671,0.008995,0.015295,0.005523,-0.008678
1947-04-01,0.002456,0.007683,0.006316,0.006385,0.010565,-0.015371,0.007177,0.005413,0.015008,0.001519,-0.011824
1947-05-01,0.002198,0.007703,0.006854,0.006385,0.009816,-0.012678,0.007170,0.009582,0.014913,0.006436,-0.011972


In [28]:
# Function to perform Volatility-Weighted Least Squares (VLS) with forward step prediction
def vls_predict_forward(train_data, test_data, var):
    n_test = len(test_data)

    # Prepare the data
    X_test = test_data[[var]].values

    vls_predictions = []

    # Perform forward step (one-step ahead) prediction
    for i in range(n_test):
        # Use data up to k for training and predict k+1
        X_train_step = train_data[[var]].iloc[:len(train_data) + i].values
        y_train_step = train_data['RET'].iloc[:len(train_data) + i].values

        # Calculate the volatility (standard deviation) for weighting
        vol_train = X_train_step.std() if len(X_train_step) > 1 else 1  # Handle very small sample sizes
        weights = 1 / (vol_train + 1e-8)  # Avoid division by zero with small epsilon

        # Initialize the Linear Regression with weighted least squares
        model = LinearRegression()

        # Fit the model using volatility as weights
        model.fit(X_train_step, y_train_step, sample_weight=weights)

        # Predict the next step
        pred = model.predict(X_test[i].reshape(1, -1))
        vls_predictions.append(pred[0])

    return vls_predictions

# Initialize a DataFrame to store VLS predictions for all variables
vls_pred_df = pd.DataFrame(index=test_data.index)

# Loop through all variables and perform VLS prediction with forward step for each
for var in variables:
    vls_predictions = vls_predict_forward(train_data, test_data, var)
    vls_pred_df[f"{var}_vls_pred"] = vls_predictions

# Display the VLS predictions DataFrame
vls_pred_df.head()

,BM_vls_pred,LTY_vls_pred,NTIS_vls_pred,INFL_vls_pred,SVAR_vls_pred,DP_vls_pred,DY_vls_pred,DFR_vls_pred,DFY_vls_pred,LTR_vls_pred,EP_vls_pred
yyyymm,,,,,,,,,,,
1947-01-01,0.004171,0.007874,0.003358,0.003180,0.009089,-0.007096,0.003358,0.005516,0.007701,0.000166,-0.012650
1947-02-01,0.004004,0.007874,0.003688,0.003180,0.010053,-0.012871,0.003347,0.007642,0.007668,0.002482,-0.019200
1947-03-01,0.002423,0.007938,0.003477,0.016245,0.008703,-0.014730,0.003362,0.007642,0.007571,0.002396,-0.023791
1947-04-01,0.001613,0.007682,0.003403,0.003180,0.008837,-0.021786,0.002099,0.002839,0.007473,-0.002494,-0.028733
1947-05-01,0.001433,0.007746,0.004654,0.003180,0.008530,-0.018417,0.002117,0.008430,0.007441,0.003511,-0.028965


In [29]:
# Function to calculate R-squared and CER for Huber and VLS predictions
def evaluate_huber_vls_performance(y_true, y_pred_huber, y_pred_vls, gamma=3):
    # R-squared for Huber
    r2_huber = r2_score(y_true, y_pred_huber)
    # R-squared for VLS
    r2_vls = r2_score(y_true, y_pred_vls)

    # Portfolio performance for Huber and VLS (CER calculation)
    cer_huber = portfolio_performance(y_pred_huber, y_true, gamma)[1]  # We are interested in CER (index 1)
    cer_vls = portfolio_performance(y_pred_vls, y_true, gamma)[1]

    return r2_huber, cer_huber, r2_vls, cer_vls

# Initialize a DataFrame to store performance results for Huber and VLS
performance_results_huber_vls = {
    "Variable": [],
    "Huber_R2": [],
    "Huber_CER": [],
    "VLS_R2": [],
    "VLS_CER": []
}

# Loop through all variables and calculate R-squared and CER for Huber and VLS
for var in variables:
    y_true = test_data['RET'].values
    y_pred_huber = huber_pred_df[f"{var}_huber_pred"].values
    y_pred_vls = vls_pred_df[f"{var}_vls_pred"].values

    # Calculate performance
    r2_huber, cer_huber, r2_vls, cer_vls = evaluate_huber_vls_performance(y_true, y_pred_huber, y_pred_vls)

    # Store results
    performance_results_huber_vls["Variable"].append(var)
    performance_results_huber_vls["Huber_R2"].append(r2_huber)
    performance_results_huber_vls["Huber_CER"].append(cer_huber)
    performance_results_huber_vls["VLS_R2"].append(r2_vls)
    performance_results_huber_vls["VLS_CER"].append(cer_vls)

# Convert results to a DataFrame for better readability
performance_df_huber_vls = pd.DataFrame(performance_results_huber_vls)

# Display the performance results for Huber and VLS
performance_df_huber_vls

,Variable,Huber_R2,Huber_CER,VLS_R2,VLS_CER
0,BM,-0.055432,0.000105,-0.014987,0.000073
1,LTY,-0.039677,0.000008,-0.452447,-0.000086
2,NTIS,0.000264,0.000059,-0.004307,0.000060
3,INFL,-0.001398,0.000046,-0.014547,0.000029
4,SVAR,0.076672,0.000229,0.055326,0.000127
5,DP,-69.374660,-0.002215,-108.999819,-0.002886
6,DY,-0.000402,0.000046,-0.007404,0.000024
7,DFR,-0.388251,-0.000003,-0.793993,-0.000036
8,DFY,-0.029802,0.000079,-0.003009,0.000044
9,LTR,-0.138909,0.000114,-0.228695,0.000108


## 稳健性检验
### 子集回归

In [40]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Split the dataset into two subsets for subset regression
train_data_1 = data.loc[:'1959-12-31']
test_data_1 = data.loc['1960-01-01':'1979-12-31']

train_data_2 = data.loc['1980-01-01':'1999-12-31']
test_data_2 = data.loc['2000-01-01':]

# Function for OLS prediction with forward step (one-step ahead)
def ols_predict(train_data, test_data, var):
    ols_predictions = []
    for i in range(len(test_data)):
        X_train = train_data[[var]].iloc[:len(train_data) + i]
        y_train = train_data['RET'].iloc[:len(train_data) + i]
        X_test = test_data[[var]].iloc[i:i+1]
        
        # OLS regression
        if len(X_train) > 1:  # Ensure there's enough data to train
            model = LinearRegression()
            model.fit(X_train, y_train)
            # Predict the next step
            pred = model.predict(X_test)
            ols_predictions.append(pred[0])
        else:
            ols_predictions.append(None)  # Append None if not enough data
    return ols_predictions

# Initialize DataFrames for storing OLS predictions
ols_pred_df_1 = pd.DataFrame(index=test_data_1.index)
ols_pred_df_2 = pd.DataFrame(index=test_data_2.index)

# Loop through variables and perform OLS on subsets
variables_without_tbl = ['BM', 'TBL', 'LTY', 'NTIS', 'INFL', 'SVAR', 'DP', 'DY', 'DFR', 'DFY', 'LTR', 'EP']
for var in variables_without_tbl:
    # OLS predictions for both subsets
    ols_pred_df_1[f"{var}_ols_pred"] = ols_predict(train_data_1, test_data_1, var)
    ols_pred_df_2[f"{var}_ols_pred"] = ols_predict(train_data_2, test_data_2, var)

# Display the OLS predictions DataFrame
ols_pred_df_1


,BM_ols_pred,TBL_ols_pred,LTY_ols_pred,NTIS_ols_pred,INFL_ols_pred,SVAR_ols_pred,DP_ols_pred,DY_ols_pred,DFR_ols_pred,DFY_ols_pred,LTR_ols_pred,EP_ols_pred
yyyymm,,,,,,,,,,,,
1960-01-01,0.011514,0.004920,-0.000544,0.007467,0.003105,0.010397,-0.018210,0.005010,0.003438,0.008190,0.009229,-0.005791
1960-02-01,0.011686,0.005051,-0.000058,0.006994,0.007231,0.010088,-0.016342,0.005034,0.007071,0.008002,0.012639,-0.004800
1960-03-01,0.010060,0.005269,0.000672,0.006611,0.005164,0.010331,-0.017664,0.005051,0.010423,0.008077,0.015530,-0.005113
1960-04-01,0.009666,0.005296,0.000064,0.006615,0.007224,0.010765,-0.017816,0.005998,-0.005887,0.008115,-0.001222,-0.005003
1960-05-01,0.010286,0.005276,0.000429,0.006901,0.005164,0.011062,-0.016428,0.005996,0.005674,0.007852,0.010712,-0.004263
...,...,...,...,...,...,...,...,...,...,...,...,...
1979-08-01,-0.003056,0.003182,-0.019428,0.010063,0.010962,0.010951,-0.045467,0.005523,-0.017971,0.006726,0.003782,-0.021325
1979-09-01,-0.003355,0.002933,-0.020239,0.010596,0.011727,0.009977,-0.047519,0.005528,-0.021709,0.006801,0.000557,-0.022215
1979-10-01,-0.005614,0.002449,-0.024575,0.010397,0.010034,0.008160,-0.049705,0.005631,-0.050557,0.006163,-0.026090,-0.023068


In [42]:
# Function for TRWLS prediction with forward step (one-step ahead)
def trwls_predict(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual):
    trwls_predictions = []
    for i in range(len(test_data)):
        X_train = train_data[[var]].iloc[:len(train_data) + i]
        y_train = train_data['RET'].iloc[:len(train_data) + i]
        X_test = test_data[[var]].iloc[i:i+1]
        
        # Calculate time and residual weights
        time_weights = calculate_time_weights(np.arange(len(X_train)), np.array([len(X_train)]), best_bandwidth_time)
        residual_weights = calculate_residual_weights(y_train - LinearRegression().fit(X_train, y_train).predict(X_train), best_bandwidth_residual)
        combined_weights = time_weights * residual_weights
        
        # TRWLS regression
        model = LinearRegression()
        model.fit(X_train, y_train, sample_weight=combined_weights)
        
        # Predict the next step
        pred = model.predict(X_test)
        trwls_predictions.append(pred[0])
    return trwls_predictions

# Initialize DataFrames for storing TRWLS predictions
trwls_pred_df_1 = pd.DataFrame(index=test_data_1.index)
trwls_pred_df_2 = pd.DataFrame(index=test_data_2.index)

# Loop through variables and perform TRWLS on subsets (using predefined best_bandwidths)
for var in variables:
    # TRWLS predictions for both subsets
    trwls_pred_df_1[f"{var}_trwls_pred"] = trwls_predict(train_data_1, test_data_1, var, best_bandwidths_time[var], best_bandwidths_residual[var])
    trwls_pred_df_2[f"{var}_trwls_pred"] = trwls_predict(train_data_2, test_data_2, var, best_bandwidths_time[var], best_bandwidths_residual[var])

# Display the TRWLS predictions DataFrame
trwls_pred_df_1

,BM_trwls_pred,LTY_trwls_pred,NTIS_trwls_pred,INFL_trwls_pred,SVAR_trwls_pred,DP_trwls_pred,DY_trwls_pred,DFR_trwls_pred,DFY_trwls_pred,LTR_trwls_pred,EP_trwls_pred
yyyymm,,,,,,,,,,,
1960-01-01,0.010515,0.007556,0.020180,0.010803,0.007514,-0.147045,-0.011260,0.007697,0.010576,0.005003,-0.054987
1960-02-01,0.010518,0.008427,0.015484,0.011296,0.001902,-0.053379,-0.010702,0.004771,0.011532,0.001384,0.013974
1960-03-01,0.010491,0.009733,0.011680,0.011049,0.006305,-0.119643,-0.010298,0.002069,0.011150,-0.001685,-0.007809
1960-04-01,0.010485,0.008644,0.011720,0.011295,0.014183,-0.127269,0.011785,0.015210,0.010959,0.016098,-0.000154
1960-05-01,0.010495,0.009298,0.014562,0.011049,0.019572,-0.057675,0.011723,0.005896,0.012297,0.003430,0.051392
...,...,...,...,...,...,...,...,...,...,...,...
1979-08-01,0.010277,-0.026271,0.045962,0.011742,0.017555,-1.513774,0.000700,0.024945,0.018033,0.010787,-1.136851
1979-09-01,0.010272,-0.027722,0.051255,0.011834,-0.000112,-1.616677,0.000814,0.027956,0.017651,0.014210,-1.198836
1979-10-01,0.010235,-0.035489,0.049286,0.011631,-0.033090,-1.726300,0.003230,0.051198,0.020902,0.042497,-1.258216


In [44]:
from sklearn.metrics import r2_score
import numpy as np

# Function to calculate portfolio returns, cumulative gain (CG), and CER
def portfolio_performance(y_pred, y_true, gamma=3):
    portfolio_returns = y_pred * y_true
    expected_return = np.mean(portfolio_returns)
    return_variance = np.var(portfolio_returns)
    cer = expected_return - (gamma / 2) * return_variance
    return cer

# Function to calculate R-squared and CER
def evaluate_model_performance(y_true, y_pred, gamma=3):
    r_squared = r2_score(y_true, y_pred)
    cer = portfolio_performance(y_pred, y_true, gamma)
    return r_squared, cer

# Initialize a DataFrame to store performance results for both OLS and TRWLS
performance_results = {
    "Variable": [],
    "OLS_R2_Subset1": [],
    "OLS_CER_Subset1": [],
    "OLS_R2_Subset2": [],
    "OLS_CER_Subset2": [],
    "TRWLS_R2_Subset1": [],
    "TRWLS_CER_Subset1": [],
    "TRWLS_R2_Subset2": [],
    "TRWLS_CER_Subset2": []
}

# Loop through each variable and calculate R-squared and CER for OLS and TRWLS on both subsets
for var in variables:
    y_true_1 = test_data_1['RET'].values
    y_true_2 = test_data_2['RET'].values
    
    # OLS predictions
    y_pred_ols_1 = ols_pred_df_1[f"{var}_ols_pred"].values
    y_pred_ols_2 = ols_pred_df_2[f"{var}_ols_pred"].values
    
    # TRWLS predictions
    y_pred_trwls_1 = trwls_pred_df_1[f"{var}_trwls_pred"].values
    y_pred_trwls_2 = trwls_pred_df_2[f"{var}_trwls_pred"].values

    # Evaluate OLS performance on both subsets
    ols_r2_1, ols_cer_1 = evaluate_model_performance(y_true_1, y_pred_ols_1)
    ols_r2_2, ols_cer_2 = evaluate_model_performance(y_true_2, y_pred_ols_2)
    
    # Evaluate TRWLS performance on both subsets
    trwls_r2_1, trwls_cer_1 = evaluate_model_performance(y_true_1, y_pred_trwls_1)
    trwls_r2_2, trwls_cer_2 = evaluate_model_performance(y_true_2, y_pred_trwls_2)

    # Store results
    performance_results["Variable"].append(var)
    performance_results["OLS_R2_Subset1"].append(ols_r2_1)
    performance_results["OLS_CER_Subset1"].append(ols_cer_1)
    performance_results["OLS_R2_Subset2"].append(ols_r2_2)
    performance_results["OLS_CER_Subset2"].append(ols_cer_2)
    performance_results["TRWLS_R2_Subset1"].append(trwls_r2_1)
    performance_results["TRWLS_CER_Subset1"].append(trwls_cer_1)
    performance_results["TRWLS_R2_Subset2"].append(trwls_r2_2)
    performance_results["TRWLS_CER_Subset2"].append(trwls_cer_2)

# Convert results to DataFrame for better readability
performance_df_subset = pd.DataFrame(performance_results)

# Display the performance results
performance_df_subset

,Variable,OLS_R2_Subset1,OLS_CER_Subset1,OLS_R2_Subset2,OLS_CER_Subset2,TRWLS_R2_Subset1,TRWLS_CER_Subset1,TRWLS_R2_Subset2,TRWLS_CER_Subset2
0,BM,-0.000424,0.000039,-0.069881,0.000061,-0.030096,0.000034,-1.098837,-0.000167
1,LTY,-0.074634,-0.000006,-0.224905,0.000119,-0.091601,0.000018,-0.296251,0.000132
2,NTIS,-0.000289,0.000035,-0.100467,0.000055,-0.374284,0.000164,-8.642627,0.000236
3,INFL,-0.029287,0.000012,-0.148701,0.000047,-0.040622,0.000035,-0.479505,0.000048
4,SVAR,-0.010967,0.000048,0.138800,0.000304,-0.394491,0.000269,0.010357,0.000367
5,DP,-0.642570,-0.000046,-0.012828,0.000008,-433.246292,-0.001665,-250.014122,-0.002870
6,DY,-0.002598,0.000020,-0.139608,0.000026,-0.002792,0.000052,-2.831770,-0.000129
7,DFR,0.022719,0.000084,-0.542554,0.000012,-0.212481,-0.000035,-0.115315,0.000092
8,DFY,-0.023397,0.000016,-0.040000,0.000043,-0.027888,0.000088,-0.457034,0.000058
9,LTR,0.068469,0.000111,-0.295485,-0.000107,-0.169243,-0.000070,-0.121301,0.000102


### 考虑不同的核函数

In [48]:
# Function for TRWLS prediction with forward step and custom kernel, with weight normalization handling
def trwls_predict_with_kernel(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual, kernel_function):
    trwls_predictions = []
    for i in range(len(test_data)):
        X_train = train_data[[var]].iloc[:len(train_data) + i]
        y_train = train_data['RET'].iloc[:len(train_data) + i]
        X_test = test_data[[var]].iloc[i:i+1]

        # Calculate time weights using the custom kernel
        time_weights = calculate_time_weights_with_kernel(np.arange(len(X_train)), np.array([len(X_train)]), best_bandwidth_time, kernel_function)
        residual_weights = calculate_residual_weights(y_train - LinearRegression().fit(X_train, y_train).predict(X_train), best_bandwidth_residual)
        
        # Combine weights
        combined_weights = time_weights * residual_weights

        # Handle the case where weights sum to zero or are too small
        if np.sum(combined_weights) == 0:
            combined_weights = np.ones_like(combined_weights)  # Use equal weights if sum is zero
        
        # Normalize the weights to avoid scaling issues
        combined_weights = combined_weights / np.sum(combined_weights)

        # TRWLS regression
        model = LinearRegression()
        model.fit(X_train, y_train, sample_weight=combined_weights)
        
        # Predict the next step
        pred = model.predict(X_test)
        trwls_predictions.append(pred[0])
    
    return trwls_predictions

# Perform TRWLS prediction using various kernels and handle weight issues
for var in variables:
    for kernel_name, kernel_function in kernels.items():
        # Perform TRWLS with each kernel
        predictions = trwls_predict_with_kernel(train_data, test_data, var, best_bandwidths_time[var], best_bandwidths_residual[var], kernel_function)
        
        # Store the predictions in the corresponding DataFrame
        kernel_pred_dfs[kernel_name][f"{var}_{kernel_name}_pred"] = predictions

# Save predictions for each kernel to CSV
for kernel_name, df in kernel_pred_dfs.items():
    df.to_csv(f"TRWLS_Predictions_with_{kernel_name.capitalize()}_Kernel.csv")
    print(f"Predictions for {kernel_name.capitalize()} kernel saved to 'TRWLS_Predictions_with_{kernel_name.capitalize()}_Kernel.csv'")



Predictions for Epanechnikov kernel saved to 'TRWLS_Predictions_with_Epanechnikov_Kernel.csv'
Predictions for Gaussian kernel saved to 'TRWLS_Predictions_with_Gaussian_Kernel.csv'
Predictions for Quantic kernel saved to 'TRWLS_Predictions_with_Quantic_Kernel.csv'
Predictions for Linear kernel saved to 'TRWLS_Predictions_with_Linear_Kernel.csv'
Predictions for Cosine kernel saved to 'TRWLS_Predictions_with_Cosine_Kernel.csv'


In [50]:
# Function to calculate portfolio returns, cumulative gain (CG), and CER
def portfolio_performance(y_pred, y_true, gamma=3):
    portfolio_returns = y_pred * y_true
    expected_return = np.mean(portfolio_returns)
    return_variance = np.var(portfolio_returns)
    cer = expected_return - (gamma / 2) * return_variance
    return cer

# Function to calculate R-squared and CER
def evaluate_model_performance(y_true, y_pred, gamma=3):
    r_squared = r2_score(y_true, y_pred)
    cer = portfolio_performance(y_pred, y_true, gamma)
    return r_squared, cer

# Initialize a DataFrame to store performance results
performance_results = {
    "Kernel": [],
    "R2": [],
    "CER": []
}

# Loop through each kernel and calculate R-squared and CER
y_true = test_data['RET'].values

for kernel_name, df in kernel_pred_dfs.items():
    for var in variables:
        y_pred = df[f"{var}_{kernel_name}_pred"].values
        
        # Calculate R² and CER
        r_squared, cer = evaluate_model_performance(y_true, y_pred)
        
        # Store results
        performance_results["Kernel"].append(f"{var}_{kernel_name}")
        performance_results["R2"].append(r_squared)
        performance_results["CER"].append(cer)

# Convert results to a DataFrame for better readability
performance_df = pd.DataFrame(performance_results)

# Display the performance results
print(performance_df)


               Kernel           R2       CER
0     BM_epanechnikov    -0.369801 -0.000085
1    LTY_epanechnikov -1932.287297 -0.018365
2   NTIS_epanechnikov    -7.226866  0.000693
3   INFL_epanechnikov    -0.016462  0.000016
4   SVAR_epanechnikov    -0.083172  0.000247
5     DP_epanechnikov  -108.999819 -0.002886
6     DY_epanechnikov    -1.246515 -0.000231
7    DFR_epanechnikov   -15.061932 -0.000513
8    DFY_epanechnikov  -163.793072  0.001481
9    LTR_epanechnikov    -3.326298  0.000148
10    EP_epanechnikov    -0.725749  0.000289
11        BM_gaussian    -1.380964  0.000221
12       LTY_gaussian   -69.646618 -0.001977
13      NTIS_gaussian    -2.557178  0.000413
14      INFL_gaussian    -0.009975  0.000091
15      SVAR_gaussian    -0.120641  0.000401
16        DP_gaussian -6485.736126 -0.047308
17        DY_gaussian    -0.092927 -0.000039
18       DFR_gaussian    -6.855048 -0.000291
19       DFY_gaussian    -1.503048  0.000271
20       LTR_gaussian    -1.519428  0.000166
21        

### 考虑不同的验证集长度

In [54]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed

# Function to calculate time weights based on kernel function (e.g., Laplace)
def calculate_time_weights_with_kernel(t, s, h, kernel_function):
    u = (t - s) / h
    return kernel_function(u)

# Function to perform cross-validation for a specific bandwidth combination
def cross_validate_bandwidth_single(train_data, var, h_time, h_residual, kernel_function, val_window, n_splits):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    mse_list = []

    for train_index, val_index in tscv.split(train_data):
        X_train, y_train = train_data[[var]].iloc[train_index], train_data['RET'].iloc[train_index]
        X_val, y_val = train_data[[var]].iloc[val_index], train_data['RET'].iloc[val_index]

        # Fit OLS to get residuals for weighting
        model_ols = LinearRegression()
        model_ols.fit(X_train, y_train)
        residuals = y_train - model_ols.predict(X_train)

        # Calculate time weights and residual weights
        time_weights = calculate_time_weights_with_kernel(np.arange(len(X_train)), np.array([len(X_train)]), h_time, kernel_function)
        residual_weights = calculate_residual_weights(residuals, h_residual)
        combined_weights = time_weights * residual_weights

        # Ensure no division by zero in weights
        combined_weights = np.clip(combined_weights, a_min=1e-8, a_max=None)

        # Fit TRWLS model with weights
        model_trwls = LinearRegression()
        model_trwls.fit(X_train, y_train, sample_weight=combined_weights)

        # Predict on validation set
        y_pred_val = model_trwls.predict(X_val)

        # Calculate validation MSE
        mse = mean_squared_error(y_val, y_pred_val)
        mse_list.append(mse)

    return np.mean(mse_list)

# Function to parallelize cross-validation for different bandwidths
def cross_validate_bandwidth(train_data, var, bandwidths_time, bandwidths_residual, kernel_function, val_window, n_splits=5):
    results = Parallel(n_jobs=-1)(delayed(cross_validate_bandwidth_single)(train_data, var, h_time, h_residual, kernel_function, val_window, n_splits)
                                  for h_time in bandwidths_time
                                  for h_residual in bandwidths_residual)

    min_mse_idx = np.argmin(results)
    best_h_time = bandwidths_time[min_mse_idx // len(bandwidths_residual)]
    best_h_residual = bandwidths_residual[min_mse_idx % len(bandwidths_residual)]
    return best_h_time, best_h_residual, np.min(results)

# Define kernel function (you can use Laplace or other kernels as needed)
def laplace_kernel(u):
    return np.exp(-np.abs(u)) / 2

# Define reduced bandwidth ranges for time and residuals (reduce size to speed up)
bandwidths_time = np.linspace(0.5, 5, 10)  # Reduced range for time weights
bandwidths_residual = np.linspace(0.5, 5, 10)  # Reduced range for residual weights

# Validation window sizes (6 months, 9 months, 15 months, 18 months)
validation_windows = [6, 9, 15, 18]

# Initialize dictionary to store the best bandwidths for each validation window
best_bandwidths = {}

# Loop through each variable and validation window, and perform cross-validation to find the best bandwidths
for var in variables_without_tbl:
    best_bandwidths[var] = {}
    for val_window in validation_windows:
        n_splits = len(train_data) // val_window  # Reduce number of splits
        best_bandwidth_time, best_bandwidth_residual, best_mse = cross_validate_bandwidth(train_data, var, bandwidths_time, bandwidths_residual, laplace_kernel, val_window, n_splits)
        best_bandwidths[var][val_window] = {"best_bandwidth_time": best_bandwidth_time, "best_bandwidth_residual": best_bandwidth_residual, "mse": best_mse}

# Display best bandwidths for each variable and validation window
best_bandwidths_df = pd.DataFrame.from_dict({(i,j): best_bandwidths[i][j] 
                                             for i in best_bandwidths.keys() 
                                             for j in best_bandwidths[i].keys()},
                                            orient='index')

print(best_bandwidths_df)


         best_bandwidth_time  best_bandwidth_residual       mse
BM   6                   5.0                      0.5  0.010512
     9                   5.0                      0.5  0.009995
     15                  5.0                      0.5  0.012338
     18                  5.0                      0.5  0.018737
TBL  6                   5.0                      0.5  0.011335
     9                   5.0                      0.5  0.010947
     15                  5.0                      5.0  0.010157
     18                  5.0                      1.5  0.008779
LTY  6                   5.0                      0.5  0.010500
     9                   5.0                      0.5  0.011369
     15                  5.0                      0.5  0.010852
     18                  5.0                      0.5  0.017086
NTIS 6                   5.0                      0.5  0.010065
     9                   5.0                      0.5  0.010327
     15                  5.0            

In [55]:
# Function for TRWLS prediction with forward step and optimal bandwidths
def trwls_predict_with_optimal_bandwidth(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual):
    trwls_predictions = []
    for i in range(len(test_data)):
        X_train = train_data[[var]].iloc[:len(train_data) + i]
        y_train = train_data['RET'].iloc[:len(train_data) + i]
        X_test = test_data[[var]].iloc[i:i+1]

        # Calculate time weights using the optimal bandwidth
        time_weights = calculate_time_weights_with_kernel(np.arange(len(X_train)), np.array([len(X_train)]), best_bandwidth_time, laplace_kernel)
        residual_weights = calculate_residual_weights(y_train - LinearRegression().fit(X_train, y_train).predict(X_train), best_bandwidth_residual)
        combined_weights = time_weights * residual_weights

        # Ensure weights are valid
        combined_weights = np.clip(combined_weights, a_min=1e-8, a_max=None)
        combined_weights = combined_weights / np.sum(combined_weights)  # Normalize the weights

        # TRWLS regression
        model = LinearRegression()
        model.fit(X_train, y_train, sample_weight=combined_weights)

        # Predict the next step
        pred = model.predict(X_test)
        trwls_predictions.append(pred[0])
    
    return trwls_predictions

# Initialize a DataFrame to store TRWLS predictions for all validation windows
trwls_pred_dfs = {
    6: pd.DataFrame(index=test_data.index),
    9: pd.DataFrame(index=test_data.index),
    15: pd.DataFrame(index=test_data.index),
    18: pd.DataFrame(index=test_data.index)
}

# Loop through each variable and validation window to perform TRWLS with optimal bandwidths
for var in variables:
    for val_window in [6, 9, 15, 18]:
        best_bandwidth_time = best_bandwidths[var][val_window]["best_bandwidth_time"]
        best_bandwidth_residual = best_bandwidths[var][val_window]["best_bandwidth_residual"]
        
        # Perform TRWLS using the optimal bandwidths
        predictions = trwls_predict_with_optimal_bandwidth(train_data, test_data, var, best_bandwidth_time, best_bandwidth_residual)
        
        # Store the predictions in the corresponding DataFrame for each window
        trwls_pred_dfs[val_window][f"{var}_trwls_pred_{val_window}"] = predictions

# Save the TRWLS predictions for each validation window to CSV files
for val_window, df in trwls_pred_dfs.items():
    df.to_csv(f"TRWLS_Predictions_{val_window}_Months.csv")
    print(f"Predictions for {val_window}-month window saved to 'TRWLS_Predictions_{val_window}_Months.csv'")


Predictions for 6-month window saved to 'TRWLS_Predictions_6_Months.csv'
Predictions for 9-month window saved to 'TRWLS_Predictions_9_Months.csv'
Predictions for 15-month window saved to 'TRWLS_Predictions_15_Months.csv'
Predictions for 18-month window saved to 'TRWLS_Predictions_18_Months.csv'


In [57]:
# Function to calculate portfolio returns, cumulative gain (CG), and CER
def portfolio_performance(y_pred, y_true, gamma=3):
    portfolio_returns = y_pred * y_true
    expected_return = np.mean(portfolio_returns)
    return_variance = np.var(portfolio_returns)
    cer = expected_return - (gamma / 2) * return_variance
    return cer

# Function to calculate R-squared and CER
def evaluate_model_performance(y_true, y_pred, gamma=3):
    r_squared = r2_score(y_true, y_pred)
    cer = portfolio_performance(y_pred, y_true, gamma)
    return r_squared, cer

# Initialize a DataFrame to store performance results for each validation window
performance_results = {
    "Variable": [],
    "Validation_Window": [],
    "R2": [],
    "CER": []
}

# Loop through each variable and validation window to calculate R² and CER
y_true = test_data['RET'].values

for val_window, df in trwls_pred_dfs.items():
    for var in variables:
        y_pred = df[f"{var}_trwls_pred_{val_window}"].values
        
        # Calculate R² and CER
        r_squared, cer = evaluate_model_performance(y_true, y_pred)
        
        # Store results
        performance_results["Variable"].append(var)
        performance_results["Validation_Window"].append(f"{val_window} Months")
        performance_results["R2"].append(r_squared)
        performance_results["CER"].append(cer)

# Convert results to a DataFrame for better readability
performance_df = pd.DataFrame(performance_results)

# Save the performance results to a CSV file
performance_df.to_csv("TRWLS_Prediction_Performance.csv", index=False)
print("Performance evaluation saved to 'TRWLS_Prediction_Performance.csv'")

# Display the performance results
print(performance_df)


Performance evaluation saved to 'TRWLS_Prediction_Performance.csv'
   Variable Validation_Window           R2       CER
0        BM          6 Months    -0.395126  0.000106
1       LTY          6 Months  -327.744998 -0.005199
2      NTIS          6 Months    -3.117175  0.000474
3      INFL          6 Months    -0.006219  0.000029
4      SVAR          6 Months    -0.089722  0.000395
5        DP          6 Months -4164.811708 -0.033358
6        DY          6 Months    -0.378869 -0.000099
7       DFR          6 Months   -11.606559 -0.000400
8       DFY          6 Months    -3.446507  0.000388
9       LTR          6 Months    -2.703108  0.000200
10       EP          6 Months   -10.170415  0.000737
11       BM          9 Months    -0.395126  0.000106
12      LTY          9 Months  -327.744998 -0.005199
13     NTIS          9 Months    -3.117175  0.000474
14     INFL          9 Months    -0.006219  0.000029
15     SVAR          9 Months    -0.322978  0.000534
16       DP          9 Months -2